# Autenticação
Primeiro precisamos autenticar nossa sessão do Colab no Google e definimos o ID do projeto

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
#Informe o id do projeto nesta linha:

project_id = "workshop-5ia-tensorflow-253922" #@param {type:"string"}


# Client Big Query

Como vamos buscar dados no bigquery, é necessário criar um cliente BG

In [0]:
# Call BigQuery and examine in dataframe
from google.cloud import bigquery

bgclient = bigquery.Client(project=project_id)

Demais imports:

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns, numpy as np
import shutil
import numpy as np
import tensorflow as tf
print(tf.__version__)

1.14.0


#Copia os dados de treinamento e teste (CSV) para diretório local

In [5]:
%pwd

'/content'

In [6]:
!gsutil cp gs://workshop-5ia-tensorflow/*.csv /content

Copying gs://workshop-5ia-tensorflow/teste.csv...
Copying gs://workshop-5ia-tensorflow/treino.csv...
- [2 files][  1.9 MiB/  1.9 MiB]                                                
Operation completed over 2 objects/1.9 MiB.                                      


In [7]:
%%bash
wc -l *.csv
head *.csv
tail *.csv

   8242 teste.csv
  24725 treino.csv
  32967 total
==> teste.csv <==
3.9689999966716343,False,33,True,1,41.0,4979697502521811334
1.6399999986247114,False,21,False,1,35.0,3095933535584005890
3.4299999971236343,True,29,True,1,39.0,1305143018446161857
2.437999997955516,False,31,True,1,33.0,7872612453343038854
2.947999997527835,True,23,True,1,39.0,1622638268154624360
3.4589999970993155,False,27,True,1,40.0,4979697502521811334
3.28499999724523,False,28,True,1,36.0,2363238223526193234
3.3399999971991075,True,21,True,1,40.0,8391424625589759186
2.4099999979789968,False,25,True,1,38.0,7420272703711713305
3.4589999970993155,True,36,True,1,37.0,7445587375556638376

==> treino.csv <==
0.949999999203339,True,36,False,1,29.0,5934265245228309013
3.2099999973081244,False,16,False,1,39.0,5937540421097454372
3.741999996861994,True,23,False,1,39.0,1002950341933487066
2.063999998269149,True,25,True,1,38.0,5107972924983092617
3.7709999968376744,True,33,True,1,40.0,411066950820961322
3.8599999967630403,True

# Cria o modelo

Determina o que será lido do CSV! (pois o mesmo está sem header)

In [0]:
CSV_COLUMNS = ['weight_kilos', 'is_male', 'mother_age', 'mother_married', 'plurality', 'gestation_weeks', 'hashmonth']
DEFAULTS =    [[0.0],          ['null'],  [0.0],        ['null'],         ['null'],    [0.0],             ['nokey']  ]
LABEL_COLUMN = 'weight_kilos'
KEY_COLUMN = 'hashmonth'

TRAIN_STEPS = 1000

Uma função inteligente, retorna a função de leitura como se fosse um lambda, de acordo com o modo (TRAIN ou !TRAIN)

In [0]:
# Define a função que lê datasets. Observe que é uma função
def read_dataset(filename, mode, batch_size=512):
  def _input_fn():
    def decode_csv(value_column):
      columns = tf.decode_csv(value_column, record_defaults=DEFAULTS)
      features = dict(zip(CSV_COLUMNS, columns))
      label = features.pop(LABEL_COLUMN)
      return features, label
    
    # Create list of files that match pattern
    file_list = tf.gfile.Glob(filename)

    # Create dataset from file list
    dataset = (tf.data.TextLineDataset(file_list)  # Read text file
                 .map(decode_csv))  # Transform each elem by applying decode_csv fn
      
    if mode == tf.estimator.ModeKeys.TRAIN:
        num_epochs = None # indefinitely
        dataset = dataset.shuffle(buffer_size=10*batch_size)
    else:
        num_epochs = 1 # end-of-input after this
 
    dataset = dataset.repeat(num_epochs).batch(batch_size)
    return dataset
  return _input_fn

Define as colunas

In [0]:
colunas_input_layer = [
    tf.feature_column.indicator_column( tf.feature_column.categorical_column_with_vocabulary_list('is_male', ['True', 'False', 'Unknown'])),
    tf.feature_column.numeric_column('mother_age'),
    tf.feature_column.indicator_column( tf.feature_column.categorical_column_with_vocabulary_list('mother_married', ['True', 'False', 'Unknown'])),
    tf.feature_column.indicator_column( tf.feature_column.categorical_column_with_vocabulary_list('plurality', ['1', '2', '3', '4', '5', '6'])),
    tf.feature_column.numeric_column('gestation_weeks')
]

In [0]:
# Cria função de camada de entrada para predição (a ser usada na avaliação)
def serving_input_fn():
    feature_placeholders = {
        'is_male': tf.placeholder(tf.string, [None]),
        'mother_age': tf.placeholder(tf.float32, [None]),
        'mother_married': tf.placeholder(tf.string, [None]),
        'plurality': tf.placeholder(tf.string, [None]),
        'gestation_weeks': tf.placeholder(tf.float32, [None])
    }
    features = {
        key: tf.expand_dims(tensor, -1)
        for key, tensor in feature_placeholders.items()
    }
    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)

In [0]:
EVAL_INTERVAL = 300
run_config = tf.estimator.RunConfig(save_checkpoints_secs = EVAL_INTERVAL, keep_checkpoint_max = 3)

In [13]:
estimator = tf.estimator.DNNRegressor(
                     model_dir = 'modelo_treinado',
                     feature_columns = colunas_input_layer,
                     hidden_units = [64, 32],
                     config = run_config)

INFO:tensorflow:Using config: {'_model_dir': 'modelo_treinado', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 300, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 3, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc075a60e48>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Cria configurações de treino e de teste

In [0]:
configuracoes_de_treino = tf.estimator.TrainSpec(
    input_fn = read_dataset('treino.csv', mode = tf.estimator.ModeKeys.TRAIN),
    max_steps = TRAIN_STEPS)

In [0]:
exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
configuracoes_de_avaliacao = tf.estimator.EvalSpec(
    input_fn = read_dataset('teste.csv', mode = tf.estimator.ModeKeys.EVAL),
    steps = None,
    start_delay_secs = 60, # start evaluating after N seconds
    throttle_secs = EVAL_INTERVAL,  # evaluate every N seconds
    exporters = exporter)

In [0]:
shutil.rmtree('modelo_treinado', ignore_errors = True) # start fresh each time

In [17]:
tf.estimator.train_and_evaluate(estimator=estimator,
                                train_spec= configuracoes_de_treino,
                                eval_spec=configuracoes_de_avaliacao)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 300.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instruction

({'average_loss': 0.22931121,
  'global_step': 1000,
  'label/mean': 3.2963123,
  'loss': 111.175476,
  'prediction/mean': 3.2749026},
 [b'modelo_treinado/export/exporter/1570142699'])

In [18]:
!ls modelo_treinado

checkpoint
eval
events.out.tfevents.1570142678.898a59e6fa5b
export
graph.pbtxt
model.ckpt-0.data-00000-of-00002
model.ckpt-0.data-00001-of-00002
model.ckpt-0.index
model.ckpt-0.meta
model.ckpt-1000.data-00000-of-00002
model.ckpt-1000.data-00001-of-00002
model.ckpt-1000.index
model.ckpt-1000.meta


Apenas para teste: Faz e exibe predições

In [19]:
prediction = estimator.predict(input_fn = read_dataset('teste.csv', mode = tf.estimator.ModeKeys.PREDICT))
for i, p in enumerate(prediction):
    print(i, p)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from modelo_treinado/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
0 {'predictions': array([3.5091376], dtype=float32)}
1 {'predictions': array([2.8692992], dtype=float32)}
2 {'predictions': array([3.4538317], dtype=float32)}
3 {'predictions': array([2.8447433], dtype=float32)}
4 {'predictions': array([3.3906605], dtype=float32)}
5 {'predictions': array([3.4072156], dtype=float32)}
6 {'predictions': array([3.0748043], dtype=float32)}
7 {'predictions': array([3.4510987], dtype=float32)}
8 {'predictions': array([3.2329304], dtype=float32)}
9 {'predictions': array([3.284788], dtype=float32)}
10 {'predictions': array([3.2829132], dtype=float32)}
11 {'predictions': array([3.2492213], dtype=float32)}
12 {'predictions': array([3.534964], dtype=float32)}
13 {'predictions': array([3.2061377], dtype

In [20]:
# Copiar o export do modelo para o Storage
!gsutil cp -r ./modelo_treinado gs://workshop-5ia-tensorflow/

Copying file://./modelo_treinado/model.ckpt-0.index [Content-Type=application/octet-stream]...
Copying file://./modelo_treinado/model.ckpt-0.data-00000-of-00002 [Content-Type=application/octet-stream]...
Copying file://./modelo_treinado/model.ckpt-1000.index [Content-Type=application/octet-stream]...
Copying file://./modelo_treinado/model.ckpt-1000.data-00001-of-00002 [Content-Type=application/octet-stream]...
- [4 files][ 25.1 KiB/ 25.1 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./modelo_treinado/events.out.tfevents.1570142678.898a59e6fa5b [Content-Type=application/octet-stream]...
Copying file://./modelo_treinado/model.ckpt-1000.meta [Content-Type=application/octet-stream]...
Copying file://./modelo_treinado/gra

# para criar um modelo com treino e teste separados!

In [0]:
shutil.rmtree('modelo_treinado', ignore_errors = True) # start fresh each time

In [0]:
estimator.train(
    input_fn = read_dataset('treino.csv', mode = tf.estimator.ModeKeys.TRAIN),
    max_steps = TRAIN_STEPS)

In [0]:
estimator.evaluate(input_fn = read_dataset('teste.csv', mode = tf.estimator.ModeKeys.EVAL),
    steps = None)

In [0]:
!ls modelo_treinado